In [10]:
import pandas as pd
import pyodbc
import sqlite3
from datetime import datetime

In [21]:
DB = {'servername': r'DESKTOP-I3QFF0F\SQLEXPRESS', 'database': 'DWH'}
now = datetime.now()
sqlite_connectie = sqlite3.connect('DWH')
export_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + ';DATABASE=' + DB['database'] + ';Trusted_Connection=yes')
export_cursor = export_conn.cursor()
export_cursor


current_time= now.strftime("%d/%m/%Y %H:%M:%S")
current_time

'21/03/2024 09:44:46'

In [22]:
staff_id = 12
satisfaction_data = {
    'SATISFACTION_SK' : [(pd.read_sql_query('SELECT MAX(SATISFACTION_SK) FROM SATISFACTION', export_conn) + 1).iloc[0, 0]],
    'SATISFACTION_YEAR_number': [2010],
    'SATISFACTION_STAFF_id': [staff_id],
    'SATISFACTION_TYPE_id': [1],
    'SATISFACTION_TYPE_description': ["goed"],
    'SALES_FSK': [(pd.read_sql_query("select MAX(SALES_SK) from SALES where SALES_STAFF_id = " + str(staff_id), export_conn)).iloc[0, 0]],
    'Changedate': [current_time]
}

dbobject = pd.DataFrame(satisfaction_data)

for index, row in dbobject.iterrows():
    query_satisfaction = f"INSERT INTO SATISFACTION VALUES ({row['SATISFACTION_SK']}, {row['SATISFACTION_YEAR_number']},'{row['SATISFACTION_STAFF_id']}', '{row['SATISFACTION_TYPE_id']}', '{row['SATISFACTION_TYPE_description']}', '{row['SALES_FSK']}', '{row['Changedate']}')"
export_cursor.execute(query_satisfaction)

export_conn.commit()

C:\Users\Alvaro\AppData\Local\Temp\ipykernel_90656\2675697234.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  'SATISFACTION_SK' : [(pd.read_sql_query('SELECT MAX(SATISFACTION_SK) FROM SATISFACTION', export_conn) + 1).iloc[0, 0]],
C:\Users\Alvaro\AppData\Local\Temp\ipykernel_90656\2675697234.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  'SALES_FSK': [(pd.read_sql_query("select MAX(SALES_SK) from SALES where SALES_STAFF_id = " + str(staff_id), export_conn)).iloc[0, 0]],


In [23]:
sales_staff_id = 12

def changeSK(tables, table, id):
    SK = (pd.read_sql_query(f'SELECT MAX({table}_SK) FROM {table}', export_conn) + 1).iloc[0, 0]
    for tbl, tbl_name in tables:
        query_change = f"UPDATE {tbl_name} SET {table}_FSK = {SK} WHERE {tbl_name}_STAFF_id = {id}"
        export_cursor.execute(query_change)
    return SK

sales_data = { 
    'SALES_SK':[changeSK([("tabel1", "SALES"),("tabel2", "SATISFACTION")], "SALES", sales_staff_id)],
    'SALES_STAFF_id':[sales_staff_id],
    'SALES_FIRST_name':["Hans"],
    'SALES_LAST_name':["Koning"],
    'SALES_EXTENSION_number':[25],
    'SALES_EMAIL_name':["@mail.com"],
    'SALES_MANAGER_id':[11],
    'SALES_BRANCH_id':[3],
    'SALES_ADDRESS1_name':["straat3"],
    'SALES_ADDRESS2_name':["straat4"],
    'SALES_POSITION_name':["boss"],
    'SALES_WORK_PHONE_number':[12345],
    'SALES_FAX_number':[12],
    'SALES_HIRED_date' : ["5/12/2024"],
    'SALES_COUNTRY_id' : [3],
    'SALES_REGION_name' : ["W"],
    'SALES_CITY_name' : ["Utrecht"],
    'SALES_POSTAL_ZONE_name' : ["2325KL"],
    'SALES_FSK' : [(pd.read_sql_query("select MAX(SALES_SK) from SALES where SALES_STAFF_id = " + str(13), export_conn)).iloc[0, 0]],
    'Changedate': [current_time]
}

dbobject2 = pd.DataFrame(sales_data)

for index, row in dbobject2.iterrows():
    query_sales = f"INSERT INTO SALES VALUES ({row['SALES_SK']},'{row['SALES_STAFF_id']}', '{row['SALES_FIRST_name']}', '{row['SALES_LAST_name']}', '{row['SALES_EXTENSION_number']}', '{row['SALES_EMAIL_name']}', '{row['SALES_MANAGER_id']}', '{row['SALES_BRANCH_id']}', '{row['SALES_ADDRESS1_name']}', '{row['SALES_ADDRESS2_name']}', '{row['SALES_POSITION_name']}', '{row['SALES_WORK_PHONE_number']}', '{row['SALES_FAX_number']}', '{row['SALES_HIRED_date']}', '{row['SALES_COUNTRY_id']}', '{row['SALES_REGION_name']}', '{row['SALES_CITY_name']}', '{row['SALES_POSTAL_ZONE_name']}', '{row['SALES_FSK']}', '{row['Changedate']}')"
export_cursor.execute(query_sales)

export_conn.commit()

C:\Users\Alvaro\AppData\Local\Temp\ipykernel_90656\2442863832.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  SK = (pd.read_sql_query(f'SELECT MAX({table}_SK) FROM {table}', export_conn) + 1).iloc[0, 0]
C:\Users\Alvaro\AppData\Local\Temp\ipykernel_90656\2442863832.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  'SALES_FSK' : [(pd.read_sql_query("select MAX(SALES_SK) from SALES where SALES_STAFF_id = " + str(13), export_conn)).iloc[0, 0]],


In [32]:
COURSE = [
    "COURSE_id",
    "COURSE_description"
]

DATE = [
    "DAY_date"
]

GO_SALES_INVENTORY = [
    "GO_SALES_INVENTORY_PRODUCT_number",
    "GO_SALES_INVENTORY_INVENTORY_YEAR",
    "GO_SALES_INVENTORY_INVENTORY_MONTH_number",
    "GO_SALES_INVENTORY_EXPECTED_VOLUME_number",
    "GO_SALES_INVENTORY_INVENTORY_COUNT_number"
]

MONTH = [
    "MONTH_number"
]

ORDER = {
    "ORDER_id"
    "ORDER_DETAILS_id"
    "ORDER_RETAILER_name"
    "ORDER_RETAILER_SITE_id"
    "ORDER_SALES_STAFF_id"
    "ORDER_SALES_BREANCH_id"
    "ORDER_date"
    "ORDER_METHOD_id"
    "ORDER_PRODUCT_id"
    "ORDER_QUANTITY_id"
    "ORDER_UNIT_COST_price"
    "ORDER_UNIT_price"
    "ORDER_UNIT_SALE_price"
    "ORDER_RETAILER_CONTACT_id"
} 

PRODUCT = [
    "PRODUCT_id"
    "PRODUCT_name"
    "PRODUCT_description"
    "PRODUCT_img"
    "PRODUCT_LINE_id"
    "PRODUCT_LINE_name"
    "PRODUCT_TYPE_id"
    "PRODUCT_INTRODUCTION_date"
    "PRODUCT_MARGIN_number"
    "PRODUCT_LANGUAGE_name"
    "PRODUCT_COST_price"
]

RETAILER = [
    "RETAILER_SITE_id"
    "RETAILER_COMPANY_name"
    "RETAILER_MR_id"
    "RETAILER_TYPE_name"
    "RETAILER_TYPE_id"
    "RETAILER_ADDRESS1_name"
    "RETAILER_ADDRESS2_name"
    "RETAILER_id"
    "RETAILER_COUNTRY_id"
    "RETAILER_REGION_name"
    "RETAILER_CITY_name"
    "RETAILER_POSTAL_ZONE_name"
    "RETAILER_ACTIVE_INDICATOR_number"
    "RETAILER_name"
    "RETAILER_FAX_number"
    "RETAILER_HEADQUARTERS_ADDRESS1_name"
    "RETAILER_HEADQUARTERS_ADDRESS2_name"
    "RETAILER_HEADQUARTERS_COUNTRY_id"
    "RETAILER_HEADQUARTERS_REGION_name"
    "RETAILER_HEADQUARTERS_CITY_name"
    "RETAILER_HEADQUARTERS_POSTAL_ZONE_name"
    "RETAILER_SEGMENT_id"
]

RETURNED_ITEM = [
    "RETURNED_ITEM_id"
    "RETURNED_ITEM_date"
    "RETURNED_ITEM_ORDER_DETAIL_id"
    ""
    ""
]


In [31]:
for index, row in dbobject2.iterrows():
    query_sales = f"INSERT INTO SALES VALUES ({row['SALES_SK']},'{row['SALES_STAFF_id']}', '{row['SALES_FIRST_name']}', '{row['SALES_LAST_name']}', '{row['SALES_EXTENSION_number']}', '{row['SALES_EMAIL_name']}', '{row['SALES_MANAGER_id']}', '{row['SALES_BRANCH_id']}', '{row['SALES_ADDRESS1_name']}', '{row['SALES_ADDRESS2_name']}', '{row['SALES_POSITION_name']}', '{row['SALES_WORK_PHONE_number']}', '{row['SALES_FAX_number']}', '{row['SALES_HIRED_date']}', '{row['SALES_COUNTRY_id']}', '{row['SALES_REGION_name']}', '{row['SALES_CITY_name']}', '{row['SALES_POSTAL_ZONE_name']}', '{row['SALES_FSK']}', '{row['Changedate']}')"
export_cursor.execute(query_sales)

export_conn.commit()

DataError: ('22018', "[22018] [Microsoft][ODBC SQL Server Driver][SQL Server]Conversion failed when converting the varchar value '       \n0  None' to data type int. (245) (SQLExecDirectW)")

In [19]:
export_conn.close()